# Imports & Connections

In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

In [2]:
DB = {
    'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',
    'database': 'johari_Herkansing_NEW'
}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Noah\AppData\Local\Temp\ipykernel_221388\2725324115.py:2: SyntaxWarning: invalid escape sequence '\S'
  'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',


In [3]:
# Local connections
sqlite_conn = sqlite3.connect('../data/processed/dwh.sqlite')
go_sales_conn = sqlite3.connect('../data/raw/go_sales.sqlite')
go_staff_conn = sqlite3.connect('../data/raw/go_staff.sqlite')
go_crm_conn = sqlite3.connect('../data/raw/go_crm.sqlite')

# Fact tables
fact_tables = ['training', 'satisfaction', 'returned_item', 'orders', 'sales_targetData', 'sales_product_forecast', 'sales_inventory_levels']
fact_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in fact_tables}

# Dimension tables
dim_tables = ['product', 'sales_staff', 'course', 'satisfaction_type', 'return_reason', 'order_method', 'retailer_contact']
dim_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in dim_tables}

# Test Tables
#training_df.loc[0:4, ['year', 'course_description']]
#product_df.head()

C:\Users\Noah\AppData\Local\Temp\ipykernel_221388\1001716418.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in fact_tables}
C:\Users\Noah\AppData\Local\Temp\ipykernel_221388\1001716418.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in dim_tables}


# Surrogate Key Creation

In [6]:
# Function to add surrogate key to dimension tables
def add_surrogate_key(conn, table_name):
    cursor = conn.cursor()
    # Add SK column
    #cursor.execute(f"ALTER TABLE {table_name} ADD SK INT IDENTITY(1,1);")
    # Add EffectiveDate column
    cursor.execute(f"ALTER TABLE {table_name} ADD EffectiveDate DATETIME NOT NULL DEFAULT GETDATE();")
    # Add EndDate column
    cursor.execute(f"ALTER TABLE {table_name} ADD EndDate DATETIME NULL;")
    # Add IsActive column
    cursor.execute(f"ALTER TABLE {table_name} ADD IsActive BIT NOT NULL DEFAULT 1;")
    conn.commit()

# Add surrogate keys to dimension tables
for table in dim_tables:
    add_surrogate_key(ssms_conn, table)

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The constraint 'PK_PRODUCT' is being referenced by table 'SALES_TARGETData', foreign key constraint 'FK_SALES_TARGETData_PRODUCT'. (3725) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Could not drop constraint. See previous errors. (3727)")

# Foreign Surrogate Key Creation

In [18]:
# Function to add surrogate foreign key columns to fact tables
def add_surrogate_foreign_key_columns(conn, fact_table, key_mappings):
    cursor = conn.cursor()
    for _, (fact_table, foreign_key_column) in key_mappings.items():
        cursor.execute(f"ALTER TABLE {fact_table} ADD {foreign_key_column} INT;")
    conn.commit()

# Add surrogate foreign key columns to fact tables
for fact_table in fact_tables:
    if fact_table == 'orders':
        key_mappings = {
            'PRODUCT_id': ('orders', 'PRODUCT_SFK'),
            'ORDER_METHOD_id': ('orders', 'ORDER_METHOD_SFK'),
            'RETAILER_CONTACT_id': ('orders', 'RETAILER_CONTACT_SFK'),
            'RETURNED_REASON': ('orders', 'RETURNED_REASON_SFK'),
        }
        #add_surrogate_foreign_key_columns(ssms_conn, fact_table, key_mappings)

# Function to populate foreign keys in fact tables
def populate_foreign_keys(conn, fact_table, dim_table, natural_key, foreign_key_column, sfk_column):
    cursor = conn.cursor()
    # Adjusted SQL statement to correctly match and update SFKs
    sql = f"""
    UPDATE {fact_table}
    SET {sfk_column} = (
        SELECT {dim_table}.SK
        FROM {dim_table}
        WHERE {dim_table}.{natural_key} = {fact_table}.{foreign_key_column}
    )
    """
    cursor.execute(sql)
    conn.commit()

# Populate Foreign Surrogate Keys

In [22]:
# Populate SFKs for the 'orders' fact table
populate_foreign_keys(ssms_conn, 'orders', 'product', 'id', 'product_id', 'PRODUCT_SFK')
populate_foreign_keys(ssms_conn, 'orders', 'order_method', 'order_method_id', 'order_method_code', 'ORDER_METHOD_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'retailer_contact', 'id', 'retailer_contact_code', 'RETAILER_CONTACT_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'RETURN_REASON', 'return_reason_id', '', 'RETURNED_REASON_SFK')

Error: ('21000', '[21000] [Microsoft][ODBC SQL Server Driver][SQL Server]Subquery returned more than 1 value. This is not permitted when the subquery follows =, !=, <, <= , >, >= or when the subquery is used as an expression. (512) (SQLExecDirectW); [21000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')

# Simulate Slowly Changing Dimensions Type 2

In [ ]:
# Step 1: Merge the Orders and Product DataFrames on the business key (ProductID)
# This operation is similar to the INNER JOIN in the SQL procedure
merged_df = pd.merge(df_orders, df_product[['ProductID', 'SK']], on='ProductID', how='inner')

# Step 2: Filter for active products
# Assuming the 'IsActive' column exists in your df_product DataFrame to indicate active products
active_products = df_product[df_product['IsActive'] == 1]

# Perform the merge again, this time ensuring we only include active products
merged_df_active = pd.merge(df_orders, active_products[['ProductID', 'SK']], on='ProductID', how='inner')

# Step 3: Update the PRODUCT_SFK in df_orders based on the SK from the active_products
# This assumes that the 'PRODUCT_SFK' column exists in your df_orders DataFrame
df_orders.update(merged_df_active[['SK']])

In [ ]:
# Close connections
ssms_conn.close()
sqlite_conn.close()
go_sales_conn.close()
go_staff_conn.close()
go_crm_conn.close()